In [1]:
import phe as paillier
import json

def storeKeys():

    public_key, private_key = paillier.generate_paillier_keypair()
    keys={}
    keys['public_key'] = {'n': public_key.n}
    print(public_key)
    keys['private_key'] = {'p': private_key.p,'q':private_key.q}
    with open('custkeys.json', 'w') as file: 
        json.dump(keys, file)
        
        
storeKeys()


<PaillierPublicKey 1df8ebd484>


In [2]:
def getKeys():
    with open('custkeys.json', 'r') as file: 
        keys=json.load(file)
        pub_key=paillier.PaillierPublicKey(n=int(keys['public_key']['n']))
        priv_key=paillier.PaillierPrivateKey(pub_key,keys['private_key']['p'],keys['private_key']['q'])
        return pub_key, priv_key 

In [3]:
def serializeData(public_key, data):
    encrypted_data_list = [public_key.encrypt(x) for x in data]
    encrypted_data={}
    encrypted_data['public_key'] = {'n': public_key.n}
    encrypted_data['values'] = [(str(x.ciphertext()), x.exponent) for x in encrypted_data_list]
    serialized = json.dumps(encrypted_data)
    return serialized

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pickle
# Load the col_tnf object from the file
with open('col_tnf.pkl', 'rb') as file:
    col_tnf = pickle.load(file)
    
col_tnf

ColumnTransformer(remainder='passthrough',
                  transformers=[('col_tnf',
                                 OneHotEncoder(drop='first', sparse=False),
                                 [0, 1, 3, 5, 6, 8])])

In [5]:
pub_key, priv_key = getKeys()
#Company	TypeName	Ram	OpSys	Weight	os	Cpu brand	ProcessorSpeed	Gpu brand	GpuModel	HDD	SSD	Hybrid	Flash_Storage	TouchScreen	IPSPanel	PPI
data = [['Dell', 'Notebook',8,'Windows 10',2.00,'Windows','Intel Core i7',2.8,'Nvidia',1050.0,0,256,0,0,0,0,141.0211998]]
trans_data = col_tnf.transform(data)

new_data = trans_data.reshape(-1)
new_data


C:\Users\agarw\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,
       1.0, 8, 2.0, 2.8, 1050.0, 0, 256, 0, 0, 0, 0, 141.0211998],
      dtype=object)

In [6]:
serializeData(pub_key, new_data)
datafile = serializeData(pub_key, new_data)
with open('data.json', 'w') as file: 
    json.dump(datafile, file)